# Neural network with multiple outputs

In [1]:
import numpy as np
import keras
from keras.layers import Dense, Input
from keras.models import Model
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
# Load the data from the 'data_processed.csv' file 
data = pd.read_csv('data_processed.csv', low_memory=False)
story_area_index = data.columns.get_loc("Story Area")
data = data[data['Number sheathing panels'].isin([1,2])]
# Split the data into X and Y
X = data.iloc[:, :story_area_index+1]  # Features from the beginning up to "Story Area"
Y = data.iloc[:, story_area_index+1:]  #
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(Y_train)

       Nail spacing [cm]  Number sheathing panels  Number end studs  \
1291                   5                        1                 1   
41082                 15                        1                 1   
54754                  5                        1                 3   
27296                 15                        1                 2   
37140                  5                        1                 1   
...                  ...                      ...               ...   
54516                  5                        1                 3   
38331                  5                        2                 1   
860                   15                        1                 1   
15968                 15                        1                 1   
56595                 15                        1                 2   

       Total number studs     Tx(s)     Ty(s)  
1291                    6  0.520351  0.522703  
41082                   5  0.693697  0.660990  
547

In [17]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras import regularizers

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model
def build_model(input_dim, num_outputs):
    inputs = Input(shape=(input_dim,))
    shared_layer = Dense(128, activation='relu')(inputs)
    shared_layer = Dense(64, activation='relu')(shared_layer)
    shared_layer = Dense(32, activation='relu')(shared_layer)

    # Output layers
    outputs = []
    for i in range(num_outputs):
        outputs.append(Dense(1, name=f'output_{i}')(shared_layer))

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Build the model
input_dimension = X_train_scaled.shape[1]
num_outputs = Y_train.shape[1]  # Assuming Y_train contains all the target outputs
model = build_model(input_dimension, num_outputs)

def lr_schedule(epoch):
    return 0.001 * 0.9 ** epoch

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
lr_callback = LearningRateScheduler(lr_schedule)
# Convert Y_train to a NumPy array
Y_train_array = Y_train.values

# Train the model
model.fit(X_train_scaled, [Y_train_array[:, i] for i in range(num_outputs)], epochs=50, batch_size=32, validation_split=0.1)

# Convert Y_test to a NumPy array
Y_test_array = Y_test.values

# Evaluate the model on the test set
evaluation = model.evaluate(X_test_scaled, [Y_test_array[:, i] for i in range(num_outputs)])

# Print the evaluation metrics
print("Evaluation Metrics:")
for i, metric_name in enumerate(model.metrics_names):
    print(f"{metric_name}: {evaluation[i]}")


Epoch 1/50
1405/1405 [==============================] - 17s 10ms/step - loss: 18.0954 - output_0_loss: 12.5217 - output_1_loss: 0.6676 - output_2_loss: 0.4501 - output_3_loss: 4.2607 - output_4_loss: 0.0998 - output_5_loss: 0.0957 - output_0_mae: 2.7463 - output_1_mae: 0.3882 - output_2_mae: 0.4771 - output_3_mae: 1.2671 - output_4_mae: 0.2165 - output_5_mae: 0.2084 - val_loss: 11.8300 - val_output_0_loss: 9.2053 - val_output_1_loss: 0.6993 - val_output_2_loss: 0.2192 - val_output_3_loss: 1.6547 - val_output_4_loss: 0.0269 - val_output_5_loss: 0.0244 - val_output_0_mae: 2.3693 - val_output_1_mae: 0.3161 - val_output_2_mae: 0.3544 - val_output_3_mae: 0.9092 - val_output_4_mae: 0.1318 - val_output_5_mae: 0.1230
Epoch 2/50
1405/1405 [==============================] - 13s 9ms/step - loss: 11.1624 - output_0_loss: 8.9619 - output_1_loss: 0.5428 - output_2_loss: 0.2037 - output_3_loss: 1.4074 - output_4_loss: 0.0242 - output_5_loss: 0.0225 - output_0_mae: 2.2848 - output_1_mae: 0.3119 - outp

KeyboardInterrupt: 

In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, Dense, Concatenate
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler



# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model
def build_model(input_dim, num_outputs):
    inputs = Input(shape=(input_dim,))
    shared_layer = Dense(128, activation='relu')(inputs)
    shared_layer = Dense(64, activation='relu')(shared_layer)
    shared_layer = Dense(32, activation='relu')(shared_layer)

    # Output layers
    outputs = []

    # Nail spacing (Classification)
    nail_spacing_output = Dense(3, activation='softmax', name='nail_spacing')(shared_layer)
    outputs.append(nail_spacing_output)

    # Number sheathing panels (Classification)
    sheathing_panels_output = Dense(2, activation='softmax', name='sheathing_panels')(shared_layer)
    outputs.append(sheathing_panels_output)

    # Number end studs (Classification)
    end_studs_output = Dense(6, activation='softmax', name='end_studs')(shared_layer)
    outputs.append(end_studs_output)

    # Total number studs (Regression)
    total_studs_output = Dense(1, name='total_studs')(shared_layer)
    outputs.append(total_studs_output)

    # Tx(s) (Regression)
    tx_output = Dense(1, name='tx')(shared_layer)
    outputs.append(tx_output)

    # Ty(s) (Regression)
    ty_output = Dense(1, name='ty')(shared_layer)
    outputs.append(ty_output)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Build the model
input_dimension = X_train_scaled.shape[1]
num_outputs = 6  # Number of outputs
model = build_model(input_dimension, num_outputs)

def lr_schedule(epoch):
    return 0.001 * 0.9 ** epoch

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, 
              loss={'nail_spacing': 'categorical_crossentropy',
                    'sheathing_panels': 'categorical_crossentropy',
                    'end_studs': 'categorical_crossentropy',
                    'total_studs': 'mse',
                    'tx': 'mse',
                    'ty': 'mse'},
              metrics={'nail_spacing': 'accuracy',
                       'sheathing_panels': 'accuracy',
                       'end_studs': 'accuracy',
                       'total_studs': 'mae',
                       'tx': 'mae',
                       'ty': 'mae'})

lr_callback = LearningRateScheduler(lr_schedule)

# Convert categorical columns to one-hot encoding with specified number of classes
def one_hot_encode_categorical(df, column_name, num_classes):
    return pd.get_dummies(df[column_name], columns=[f'{column_name}_{i}' for i in range(num_classes)])

num_nail_spacing_classes = 3
num_sheathing_panels_classes = 2
num_end_studs_classes = 6

Y_train_categorical = {
    'nail_spacing': one_hot_encode_categorical(Y_train, 'Nail spacing [cm]', num_nail_spacing_classes),
    'sheathing_panels': one_hot_encode_categorical(Y_train, 'Number sheathing panels', num_sheathing_panels_classes),
    'end_studs': one_hot_encode_categorical(Y_train, 'Number end studs', num_end_studs_classes)
}

Y_test_categorical = {
    'nail_spacing': one_hot_encode_categorical(Y_test, 'Nail spacing [cm]', num_nail_spacing_classes),
    'sheathing_panels': one_hot_encode_categorical(Y_test, 'Number sheathing panels', num_sheathing_panels_classes),
    'end_studs': one_hot_encode_categorical(Y_test, 'Number end studs', num_end_studs_classes)
}

# Train the model
model.fit(X_train_scaled, [Y_train_categorical[key] for key in Y_train_categorical], epochs=50, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
evaluation = model.evaluate(X_test_scaled, [Y_test_categorical[key] for key in Y_test_categorical])

# Print the evaluation metrics
print("Evaluation Metrics:")
for output_name in model.output_names:
    print(f"{output_name}: {evaluation[model.output_names.index(output_name)]}")


Epoch 1/50


ValueError: in user code:

    File "c:\Users\anejj\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\anejj\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\anejj\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\anejj\anaconda3\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\anejj\anaconda3\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "c:\Users\anejj\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "c

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the categorical outputs
nail_spacing_categories = [5, 10, 15]
number_sheathing_categories = [1, 2]
number_end_studs_categories = list(range(1, 7))

# Convert Y_train to a DataFrame if it's not already
Y_train_df = pd.DataFrame(Y_train, columns=['Nail spacing [cm]', 'Number sheathing panels', 'Number end studs', 'Total number studs', 'Tx(s)', 'Ty(s)'])

# Convert 'Nail spacing [cm]' to categorical values with labels starting from 0
Y_train_df['Nail spacing [cm]'] = pd.Categorical(Y_train_df['Nail spacing [cm]'])
Y_train_df['Nail spacing [cm]'] = Y_train_df['Nail spacing [cm]'].cat.codes

# Convert 'Number sheathing panels' to categorical values with labels starting from 0
Y_train_df['Number sheathing panels'] = pd.Categorical(Y_train_df['Number sheathing panels'])
Y_train_df['Number sheathing panels'] = Y_train_df['Number sheathing panels'].cat.codes

Y_train_df['Number end studs'] = pd.Categorical(Y_train_df['Number end studs'])
Y_train_df['Number end studs'] = Y_train_df['Number end studs'].cat.codes

# Transform the categorical outputs to one-hot encoding
Y_train_nail_spacing = to_categorical(Y_train_df['Nail spacing [cm]'], num_classes=len(nail_spacing_categories))
Y_train_number_sheathing = to_categorical(Y_train_df['Number sheathing panels'], num_classes=len(number_sheathing_categories))
Y_train_number_end_studs = to_categorical(Y_train_df['Number end studs'] - 1, num_classes=len(number_end_studs_categories))

#Define the input layer
input_layer = Input(shape=(X_train.shape[1],))
hidden_layer_1 = Dense(64, activation='relu')(input_layer)
hidden_layer_2 = Dense(32, activation='relu')(hidden_layer_1)

# Output layers for each output
nail_spacing_output = Dense(len(nail_spacing_categories), activation='softmax', name='nail_spacing')(hidden_layer_2)
number_sheathing_output = Dense(len(number_sheathing_categories), activation='softmax', name='number_sheathing')(hidden_layer_2)
number_end_studs_output = Dense(len(number_end_studs_categories), activation='softmax', name='number_end_studs')(hidden_layer_2)
total_number_studs_output = Dense(1, name='total_number_studs')(hidden_layer_2)
Tx_output = Dense(1, name='Tx')(hidden_layer_2)
Ty_output = Dense(1, name='Ty')(hidden_layer_2)

# Create the model
model = Model(inputs=input_layer, outputs=[nail_spacing_output, number_sheathing_output, number_end_studs_output,
                                           total_number_studs_output, Tx_output, Ty_output])


# Compile the model
model.compile(optimizer='adam', loss={
    'nail_spacing': 'categorical_crossentropy',
    'number_sheathing': 'categorical_crossentropy',
    'number_end_studs': 'categorical_crossentropy',
    'total_number_studs': 'mean_squared_error',
    'Tx': 'mean_squared_error',
    'Ty': 'mean_squared_error'
}, metrics={
    'nail_spacing': 'accuracy',
    'number_sheathing': 'accuracy',
    'number_end_studs': 'accuracy',
    'total_number_studs': 'mean_absolute_error',
    'Tx': 'mean_absolute_error',
    'Ty': 'mean_absolute_error'
})

print(model.output_names)
# Train the model
model.fit(X_train_scaled, {
    'nail_spacing': Y_train_nail_spacing,
    'number_sheathing': Y_train_number_sheathing,
    'number_end_studs': Y_train_number_end_studs,
    'total_number_studs': Y_train_df['Total number studs'],
    'Tx': Y_train_df['Tx(s)'],
    'Ty': Y_train_df['Ty(s)']
}, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
predictions = model.predict(X_test_scaled)

# For each output, you can extract the predictions
nail_spacing_pred = [nail_spacing_categories[i] for i in predictions[0].argmax(axis=1)]
number_sheathing_pred = [number_sheathing_categories[i] for i in predictions[1].argmax(axis=1)]
number_end_studs_pred = predictions[2].argmax(axis=1) + 1
total_number_studs_pred = predictions[3][:, 0]
Tx_pred = predictions[4][:, 0]
Ty_pred = predictions[5][:, 0]

# Convert number_end_studs_pred to integer (if it's not already)
number_end_studs_pred = number_end_studs_pred.astype(int)

# Calculate Mean Squared Error for each output
mse_nail_spacing = mean_squared_error(Y_test['Nail spacing [cm]'], nail_spacing_pred)
mse_number_sheathing = mean_squared_error(Y_test['Number sheathing panels'], number_sheathing_pred)
mse_number_end_studs = mean_squared_error(Y_test['Number end studs'], number_end_studs_pred)
mse_total_number_studs = mean_squared_error(Y_test['Total number studs'], total_number_studs_pred)
mse_Tx = mean_squared_error(Y_test['Tx(s)'], Tx_pred)
mse_Ty = mean_squared_error(Y_test['Ty(s)'], Ty_pred)


print(f'Mean Squared Error - Nail Spacing: {mse_nail_spacing}')
print(f'Mean Squared Error - Number Sheathing: {mse_number_sheathing}')
print(f'Mean Squared Error - Number End Studs: {mse_number_end_studs}')
print(f'Mean Squared Error - Total Number Studs: {mse_total_number_studs}')
print(f'Mean Squared Error - Tx: {mse_Tx}')
print(f'Mean Squared Error - Ty: {mse_Ty}')


['nail_spacing', 'number_sheathing', 'number_end_studs', 'total_number_studs', 'Tx', 'Ty']
Epoch 1/50
1246/1246 [==============================] - 11s 7ms/step - loss: 9.0096 - nail_spacing_loss: 0.8062 - number_sheathing_loss: 0.3500 - number_end_studs_loss: 1.0298 - total_number_studs_loss: 6.6231 - Tx_loss: 0.1183 - Ty_loss: 0.0822 - nail_spacing_accuracy: 0.6680 - number_sheathing_accuracy: 0.8619 - number_end_studs_accuracy: 0.5878 - total_number_studs_mean_absolute_error: 1.4756 - Tx_mean_absolute_error: 0.2232 - Ty_mean_absolute_error: 0.1960 - val_loss: 3.1640 - val_nail_spacing_loss: 0.6272 - val_number_sheathing_loss: 0.2665 - val_number_end_studs_loss: 0.6059 - val_total_number_studs_loss: 1.6186 - val_Tx_loss: 0.0264 - val_Ty_loss: 0.0194 - val_nail_spacing_accuracy: 0.7402 - val_number_sheathing_accuracy: 0.8850 - val_number_end_studs_accuracy: 0.7676 - val_total_number_studs_mean_absolute_error: 0.8705 - val_Tx_mean_absolute_error: 0.1273 - val_Ty_mean_absolute_error: 0.1

InvalidIndexError: (slice(None, None, None), 0)